In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import urllib
from PIL import ImageFile

In [2]:
base_url = 'https://www.schwr.com/article/category/szydt-'
page_num = 9
year_above = '2016'
keyword = '月份水量实况'
output_path = 'hydro_pdfs/' #change to the location you want to store the pdfs 

In [3]:
def get_table_links_one_page(url, keyword = '月份水量实况', year_above = '2016'):
    links_dict = {}
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    links = soup.find_all('a')
    for link in links:
        try:
            if keyword in link.text:
                year_ = re.findall(r'\d+', link.text)[0]
                if year_ >= year_above:
                    links_dict[link.text] = 'https://www.schwr.com'+link['href']
        except:
            continue
    return links_dict

def get_table_links_all_page(base_url, page_num, keyword = '月份水量实况', year_above = '2016'):
    links_all_page = {}
    for i in range(page_num):
        url = base_url+str(i+1)
        print(url)
        links_one_page = get_table_links_one_page(url, keyword = keyword)
        links_all_page.update(links_one_page)
    return links_all_page

In [9]:
all_links = get_table_links_all_page(base_url, page_num, keyword)

https://www.schwr.com/article/category/szydt-1
https://www.schwr.com/article/category/szydt-2
https://www.schwr.com/article/category/szydt-3
https://www.schwr.com/article/category/szydt-4
https://www.schwr.com/article/category/szydt-5
https://www.schwr.com/article/category/szydt-6
https://www.schwr.com/article/category/szydt-7
https://www.schwr.com/article/category/szydt-8
https://www.schwr.com/article/category/szydt-9


In [12]:
def download_images(url, output_path, name, size_threshold = 10000):
    """
    download images that have size over threshold from webpage
    """
    size_threshold = int(size_threshold)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

     #get all image url from the webpage
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    image_urls = []
    for i in soup.find_all('img'):
        try:
            image_urls.append(i['original'])
        except:
            image_urls.append(i['src'])

    i = 0
    for image_url in image_urls:
        try:
            file = urllib.request.urlopen(image_url)
            img_size = file.headers.get("content-length")
            img_size = int(img_size)
        except:
            continue
        if img_size >= size_threshold: # filter out irrelevant images
            urllib.request.urlretrieve(image_url, os.path.join(output_path,name+'.jpg'))
            i += 1
    print('Images successfully saved at ' + output_path)

In [13]:
for name, url in all_links.items():
    download_images(url, output_path, name, size_threshold=0)

Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfully saved at hydro_pdfs/
Images successfu

In [ ]:
#install if not already
#!pip install tabula-py pytesseract

In [4]:
import os
import tabula
import pytesseract
import pandas as pd
import PIL

In [5]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
# Set the path to the input folder containing the images
input_folder = output_path

# Set the path to the output folder to store the Excel files
output_folder = output_path

# Loop over all the image files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Set the path to the input image file
        input_image = os.path.join(input_folder, filename)

        # Extract text from the image using pytesseract
        text = pytesseract.image_to_string(PIL.Image.open(input_image), lang='chi_sim')

        # Find the table in the text using tabula
        tables = tabula.read_pdf(input_image, pages="all")

        # Convert the table to a Pandas DataFrame
        df = pd.DataFrame(tables[0])

        # Set the path to the output Excel file
        output_excel = os.path.join(output_folder, os.path.splitext(filename)[0] + ".xlsx")

        # Write the DataFrame to the output Excel file
        df.to_excel(output_excel, index=False)

In [6]:
text = pytesseract.image_to_string(PIL.Image.open('hydro_pdfs/四川省2023年二月份水量实况image_0.jpg'), lang='chi_sim')

In [8]:
text

'2023 年 2 月四川省主要江河水量及与常年同期比较情况表\n\n来水量                     同比\n月份            河流名称\n(亿ma)                    (3%)\n雅车江               28.70                  50.3\n安宁河                2.46                    7.5\n大渡河       21.38        107.9\n青衣江                487                   58.0\n2月                岷江                    29.85                      28.6\n沱江                3.18                 67.1\n党江        3.33         19.0\n嘉陵江                      11.04                         75.8\n渠江                3.80                 41.3\n\n'

In [7]:
df = pd.read_html(text)[0]

ValueError: No tables found